##### Generar "results_group_movie_country"

In [0]:
dbutils.widgets.text("file_date","2024-12-30")
file_date = dbutils.widgets.get("file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
df_movies = spark.read.format("delta").load(f"{silver_folder_path}/movies").filter(f"file_date = '{file_date}'")
df_production_countries = spark.read.format("delta").load(f"{silver_folder_path}/production_countries").filter(f"file_date = '{file_date}'")
df_countries = spark.read.format("delta").load(f"{silver_folder_path}/countries")

In [0]:
df_movie_country = df_countries.join(df_production_countries,df_countries.country_id==df_production_countries.country_id,"inner")

In [0]:
df_movie_full = df_movies.join(df_movie_country,df_movies.movie_id==df_movie_country.movie_id,"inner").select("budget","revenue","year_release_date","country_name")

In [0]:
from pyspark.sql.functions import sum

df_movie_group = df_movie_full.groupBy("year_release_date","country_name")\
.agg(
    sum("budget").alias("total_budget"),
    sum("revenue").alias("total_revenue")
)

#### Partition

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc,dense_rank,lit
dense_rank_movie = Window.partitionBy("year_release_date").orderBy(desc("total_budget"),desc("total_revenue"))
df_final = df_movie_group.withColumn("dense_rank",dense_rank().over(dense_rank_movie)).withColumn("created_date",lit(file_date))

##### Guardar el resultado en Gold

In [0]:
merge_condition="target.year_release_date = source.year_release_date  AND target.country_name = source.country_name AND target.created_date = source.created_date"
merge_delta_lake(df_final,"movie_gold","results_group_movie_country",gold_folder_path,merge_condition,"created_date")

In [0]:
%sql
SELECT * FROM movie_gold.results_group_movie_country